In [1]:
import pandas as pd
import json

from datetime import datetime

In [2]:
def columna_list(lista):
    lista = lista.replace("\"", "")
    lista = lista.strip("[]") 
    return lista

def columna_list2(lista):
    if lista == []:
        return ""
    return ",".join(lista)

def to_dict(string):
    if string != "[]":
        string = json.loads(string.replace("'", "\""))
        return ",".join([s["screen_name"] for s in string])
    return ""

def to_list(list_):
    if list_ != "[]":
        list_ = list_[1:-1]
        list_ = list_.split(",")
        return ",".join([s.strip().strip("'") for s in list_])
    return ""

In [3]:
%%time
df = pd.read_csv("C:/Users/Daniel/Desktop/proyecto/notebooks/ya en github/twint + trends_marzo/trends_marzo.csv", 
                 sep = ";")

<string>:2: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.


Wall time: 3min 44s


In [4]:
#df.info()

In [5]:
df.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest', 'trend'],
      dtype='object')

In [6]:
columns_to_drop = ["conversation_id", "cashtags", "timezone", "user_id", "name", "near", "geo", "source",
                   "user_rt_id", "user_rt", "retweet_id", "retweet_date", "translate", "trans_src",
                   "trans_dest", "place", "quote_url", "thumbnail", "created_at", "id", "link"]

df.drop(columns_to_drop, axis = 1, inplace = True)

df = df[df.language == "es"]

df.drop("language", axis = 1, inplace = True)

df = df.reset_index().drop("index", axis = 1)

In [7]:
#df.head()

In [8]:
#df.info()

In [9]:
df.columns[df.isna().all()]

Index([], dtype='object')

In [10]:
df.photos = df.photos.apply(lambda x : 1 if x != "[]" else 0)

In [11]:
df.retweet = df.retweet.apply(lambda x : 1 if x == "True" else 0)

In [12]:
df.urls = df.urls.apply(lambda x : 1 if x != "[]" else 0)

In [13]:
%%time
df.date = df.date.apply(lambda x : datetime.strptime(x, "%Y-%m-%d"))

Wall time: 2min 4s


In [14]:
%%time
df.time = df.time.apply(lambda x : datetime.strptime(x, "%H:%M:%S"))

Wall time: 2min 9s


In [15]:
for col in df.columns:
    if df[col].dtype == "int64":
        df[col] = df[col].astype("int32")

In [16]:
reply_to_rows = []
for num, row in enumerate(df.reply_to):
    try:
        to_dict(row)
    except:
        reply_to_rows.append(num)

In [17]:
df.drop(reply_to_rows, inplace = True)

In [18]:
df.reply_to = df.reply_to.apply(to_dict)

In [19]:
df = df.reset_index().drop("index", axis = 1)

In [20]:
mention_rows = []
for num, row in enumerate(df.mentions):
    try:
        to_dict(row)
    except:
        mention_rows.append(num)

In [21]:
df.drop(mention_rows, inplace = True)

In [22]:
df.mentions = df.mentions.apply(to_dict)

In [23]:
df = df.reset_index().drop("index", axis = 1)

In [24]:
hashtags_rows = []
for num, row in enumerate(df.hashtags):
    try:
        to_list(row)
    except:
        hashtags_rows.append(num)

In [25]:
len(hashtags_rows)

0

In [26]:
df.drop(hashtags_rows, inplace = True)

In [27]:
df.hashtags = df.hashtags.apply(to_list)

In [28]:
df = df.reset_index().drop("index", axis = 1)

In [29]:
df

date                time         username  \
0       2020-03-12 1900-01-01 00:58:27    4wheelshop4x4   
1       2020-03-11 1900-01-01 23:24:24     arishkkacaro   
2       2020-03-11 1900-01-01 23:22:23         m_ontoba   
3       2020-03-11 1900-01-01 23:12:20    silviavazquez   
4       2020-03-11 1900-01-01 22:55:30  mikesupertramp_   
...            ...                 ...              ...   
2061645 2020-03-25 1900-01-01 08:25:55    sergio_lorduk   
2061646 2020-03-25 1900-01-01 08:14:23           lonoak   
2061647 2020-03-25 1900-01-01 06:13:39     rcuquerella1   
2061648 2020-03-25 1900-01-01 02:50:47     sergio_camps   
2061649 2020-03-25 1900-01-01 02:09:18    sergio_ssddcc   

                                                     tweet  \
0        Prepárate para Descubrir esta Noche un Vehícul...   
1        @T13 @Rodrigo40946868 Creo que quieren ver ard...   
2        @JuegaMundoplay @wastewin7 @iloveclicks 🙏🏻 tri...   
3        Ya han pasado #16años de ese trágico y fatídic...   
4          Literalmente en Stgo esta la pura caga #11M2020   
...                                                    ...   
2061645  Vamos a darle un poco de ritmo a este día, áni...   
2061646                  @Sergio_deLuz Mucho ánimo, Sergio   
2061647  @SergioVanoG @Nopanaden @RaquelBlascoR @tdelac...   
2061648  Echando una mano en lo que se puede! Animo! en...   
2061649  @SebasOsio Pues me has dado aun mas interes. Y...   

                            mentions  urls  photos  replies_count  \
0        4wheelshop4x4,4wheelshop4x4     1       1              0   
1                                        0       0              0   
2                                        0       0              0   
3                                        0       1              0   
4                                        0       0              0   
...                              ...   ...     ...            ...   
2061645                                  1       0              0   
2061646                                  0       0              0   
2061647                                  0       0              0   
2061648                                  1       0              0   
2061649                                  0       0              0   

         retweets_count  likes_count  \
0                     0            0   
1                     0            0   
2                     0            3   
3                    16            5   
4                     0            0   
...                 ...          ...   
2061645               0            1   
2061646               0            0   
2061647               0            1   
2061648               0            1   
2061649               0            2   

                                                  hashtags  retweet  video  \
0                            11m2020,motor,pureperformance        0      1   
1                     11m2020,elpeorpresidentedelahistoria        0      0   
2                                                  11m2020        0      0   
3        16años,noolvidamos,niolvidamosniperdonamos,11m...        0      1   
4                                                  11m2020        0      0   
...                                                    ...      ...    ...   
2061645                                                           0      0   
2061646                                                           0      0   
2061647                                                           0      0   
2061648                                                           0      0   
2061649                                                           0      0   

                                                  reply_to         trend  
0                                                               #11M2020  
1                                      T13,Rodrigo40946868      #11M2020  
2                     JuegaMundoplay,wastewin7,iloveclicks      #11M2020  
3            

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2061650 entries, 0 to 2061649
Data columns (total 15 columns):
 #   Column          Dtype         
---  ------          -----         
 0   date            datetime64[ns]
 1   time            datetime64[ns]
 2   username        object        
 3   tweet           object        
 4   mentions        object        
 5   urls            int32         
 6   photos          int32         
 7   replies_count   int32         
 8   retweets_count  int32         
 9   likes_count     int32         
 10  hashtags        object        
 11  retweet         int32         
 12  video           int32         
 13  reply_to        object        
 14  trend           object        
dtypes: datetime64[ns](2), int32(7), object(6)
memory usage: 180.9+ MB


In [31]:
df.to_csv("tweets_marzo.csv", sep = ";", index = False)

In [32]:
df.shape

(2061650, 15)